In [ ]:
!pip install transformers==4.17
# !pip install optuna
!pip install datasets
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 45.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48

In [ ]:
from transformers import Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time
from torchinfo import summary
from tabulate import tabulate

**Dataset Utilization provided by NLBSE**

In [ ]:
splits = {'java_train': 'data/java_train-00000-of-00001.parquet', 'java_test': 'data/java_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_test"])

display(train_df.head())
display(test_df.head())
display(train_df.shape)
display(test_df.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,Abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,Abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | Ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,AbstractContractGetFileStatusTest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,AbstractContractGetFileStatusTest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,AbstractContractGetFileStatusTest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,AbstractContractGetFileStatusTest.java,accept everything.,1,accept everything. | AbstractContractGetFileSt...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,AbstractContractGetFileStatusTest.java,accept nothing.,1,accept nothing. | AbstractContractGetFileStatu...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,AbstractContractGetFileStatusTest.java,equals the @code match field.,1,equals the @code match field. | AbstractContra...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,AbstractS3ACommitterFactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,ApplicationConstants.java,environment for applications.,1,environment for applications. | ApplicationCon...,"[0, 0, 1, 0, 0, 0, 0]"


(7614, 6)

(1725, 6)

In [ ]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,abstractcontractgetfilestatustest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,abstractcontractgetfilestatustest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,abstractcontractgetfilestatustest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,abstractcontractgetfilestatustest.java,accept everything.,1,accept everything | abstractcontractgetfilesta...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,abstractcontractgetfilestatustest.java,accept nothing.,1,accept nothing | abstractcontractgetfilestatus...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,abstractcontractgetfilestatustest.java,equals the @code match field.,1,equals the code match field | abstractcontract...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,abstracts3acommitterfactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,applicationconstants.java,environment for applications.,1,environment for applications | applicationcons...,"[0, 0, 1, 0, 0, 0, 0]"


In [ ]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [ ]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    # per-category precision, recall, and F1
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)

    # per-category accuracy
    accuracy_per_label = (preds == labels).mean(axis=0)

    # Convert results into a dictionary
    metrics = {
        'accuracy_per_label': accuracy_per_label.tolist(),
        'precision_per_label': precision.tolist(),
        'recall_per_label': recall.tolist(),
        'f1_per_label': f1.tolist()
    }

    # overall accuracy
    overall_accuracy = (preds == labels).all(axis=1).mean()
    metrics['overall_accuracy'] = overall_accuracy

    return metrics

In [ ]:
# def objective(trial):
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
#     batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size * 2,
#         warmup_steps=10,
#         weight_decay=weight_decay,
#         evaluation_strategy="epoch",
#         learning_rate=learning_rate,
#         gradient_accumulation_steps=2,
#         report_to="none",
#     )

#     trainer = OneHotTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()
#     eval_results = trainer.evaluate()

#     return eval_results["eval_f1"]

In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# print("Best trial:")
# print(study.best_trial)

In [ ]:
# best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=6,  # Updated value
    per_device_train_batch_size=16,  # Updated value
    per_device_eval_batch_size=16 * 2,  # Derived from updated batch_size
    learning_rate=1.4e-05,  # Updated value
    weight_decay=0.0030,  # Updated value
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)


trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time

print("=====================Training Time=====================")
print(f"Training time: {training_time:.2f} seconds")
print("=======================================================")

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6091
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1140


Epoch,Training Loss,Validation Loss,Accuracy Per Label,Precision Per Label,Recall Per Label,F1 Per Label,Overall Accuracy
0,No log,0.216065,"[0.845042678923178, 0.9940906106369009, 0.9231779382797111, 0.9067629678266579, 0.9625738673670388, 0.9868680236375574, 0.9612606697307945]","[0.7779097387173397, 0.9772727272727273, 0.0, 0.9289940828402367, 0.9562043795620438, 0.0, 0.0]","[0.9303977272727273, 0.8431372549019608, 0.0, 0.7268518518518519, 0.7197802197802198, 0.0, 0.0]","[0.8473479948253557, 0.9052631578947369, 0.0, 0.8155844155844156, 0.8213166144200627, 0.0, 0.0]",0.739330
1,No log,0.155723,"[0.896913985554826, 0.9986868023637557, 0.9231779382797111, 0.9409061063690085, 0.9684832567301379, 0.9868680236375574, 0.9612606697307945]","[0.8501920614596671, 0.9803921568627451, 0.0, 0.8958333333333334, 0.91875, 0.0, 0.0]","[0.9431818181818182, 0.9803921568627451, 0.0, 0.8958333333333334, 0.8076923076923077, 0.0, 0.0]","[0.8942760942760942, 0.9803921568627451, 0.0, 0.8958333333333334, 0.8596491228070176, 0.0, 0.0]",0.803677
2,0.233000,0.133245,"[0.9139855548260013, 0.9986868023637557, 0.9231779382797111, 0.9415627051871307, 0.9717662508207485, 0.9868680236375574, 0.9612606697307945]","[0.8855989232839838, 0.9803921568627451, 0.0, 0.8802660753880266, 0.9426751592356688, 0.0, 0.0]","[0.9346590909090909, 0.9803921568627451, 0.0, 0.9189814814814815, 0.8131868131868132, 0.0, 0.0]","[0.9094678645473393, 0.9803921568627451, 0.0, 0.89920724801812, 0.8731563421828908, 0.0, 0.0]",0.806303
3,0.233000,0.120225,"[0.9185817465528562, 0.9986868023637557, 0.9271175311884439, 0.9494418910045962, 0.9737360472751149, 0.9875246224556796, 0.963887065003283]","[0.8918918918918919, 0.9803921568627451, 0.625, 0.9080459770114943, 0.9329268292682927, 1.0, 0.6666666666666666]","[0.9375, 0.9803921568627451, 0.1282051282051282, 0.9143518518518519, 0.8406593406593407, 0.05, 0.13559322033898305]","[0.9141274238227147, 0.9803921568627451, 0.2127659574468085, 0.9111880046136102, 0.884393063583815, 0.09523809523809523, 0.22535211267605634]",0.821405
4,0.233000,0.117391,"[0.9198949441891004, 0.9980302035456337, 0.9284307288246881, 0.9507550886408405, 0.9770190413657256, 0.9875246224556796, 0.9632304661851608]","[0.886968085106383, 0.9615384615384616, 0.6818181818181818, 0.9084668192219679, 0.9454545454545454, 1.0, 0.7142857142857143]","[0.9474431818181818, 0.9803921568627451, 0.1282051282051282, 0.9189814814814815, 0.8571428571428571, 0.05, 0.0847457627118644]","[0.9162087912087912, 0.970873786407767, 0.2158273381294964, 0.9136939010356732, 0.899135446685879, 0.09523809523809523, 0.15151515151515152]",0.826658
5,0.102700,0.116665,"[0.927774130006566, 0.9980302035456337, 0.927774130006566, 0.9500984898227183, 0.9783322390019698, 0.9875246224556796, 0.963887065003283]","[0.8949468085106383, 0.9615384615384616, 0.6296296296296297, 0.906392694063927, 0.9461077844311377, 1.0, 0.75]","[0.9559659090909091, 0.9803921568627451, 0.1452991452991453, 0.9189814814814815, 0.8681318681318682, 0.05, 0.1016949152542373]","[0.9244505494505495, 0.970873786407767, 0.2361111111111111, 0.9126436781609195, 0.9054441260744985, 0.09523809523809523, 0.1791044776119403]",0.831911


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****


=====================Training Time=====================
Training time: 848.56 seconds


In [ ]:
!mkdir 'best_model_distilbert_java'
!mkdir 'best_model_distilbert_java_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_distilbert_java')
tokenizer.save_pretrained('./best_model_distilbert_java_tokenizer')

# Load model and tokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('./best_model_distilbert_java')
tokenizer = DistilBertTokenizer.from_pretrained('./best_model_distilbert_java_tokenizer')

Saving model checkpoint to ./best_model_distilbert_java
Configuration saved in ./best_model_distilbert_java/config.json
Model weights saved in ./best_model_distilbert_java/pytorch_model.bin
tokenizer config file saved in ./best_model_distilbert_java_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_distilbert_java_tokenizer/special_tokens_map.json
loading configuration file ./best_model_distilbert_java/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_

## Model Testing

In [ ]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [ ]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [ ]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

# Display metrics and inference time
print("=================Evaluation Metrics================")
print("Evaluation Metrics:", metrics)
print("===================================================")

print("=================Inference Time================")
print(f"Inference Time: {inference_time:.2f} seconds")
print("===================================================")

***** Running Prediction *****
  Num examples = 1725
  Batch size = 32


=================Evaluation Metrics================
Evaluation Metrics: {'accuracy_per_label': [0.8660869565217392, 1.0, 0.9356521739130435, 0.9304347826086956, 0.9617391304347827, 0.9942028985507246, 0.9565217391304348], 'precision_per_label': [0.8557588805166846, 1.0, 0.36363636363636365, 0.9102902374670184, 0.7681818181818182, 1.0, 0.18181818181818182], 'recall_per_label': [0.8912556053811659, 1.0, 0.11764705882352941, 0.8004640371229699, 0.9184782608695652, 0.3333333333333333, 0.029411764705882353], 'f1_per_label': [0.8731466227347611, 1.0, 0.17777777777777778, 0.8518518518518519, 0.8366336633663366, 0.5, 0.05063291139240506], 'overall_accuracy': 0.7872463768115942}
=================Inference Time================
Inference Time: 13.69 seconds


In [ ]:
!zip -r best_model_distilbert_java.zip './best_model_distilbert_java'
!zip -r best_model_distilbert_java_tokenizer.zip './best_model_distilbert_java_tokenizer'

  adding: best_model_distilbert_java/ (stored 0%)
  adding: best_model_distilbert_java/training_args.bin (deflated 51%)
  adding: best_model_distilbert_java/pytorch_model.bin (deflated 8%)
  adding: best_model_distilbert_java/config.json (deflated 53%)
  adding: best_model_distilbert_java_tokenizer/ (stored 0%)
  adding: best_model_distilbert_java_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_distilbert_java_tokenizer/tokenizer_config.json (deflated 43%)
  adding: best_model_distilbert_java_tokenizer/vocab.txt (deflated 53%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_distilbert_java.zip "/content/drive/MyDrive/FYP/Revised/Models/DistilBert_java/"
!cp best_model_distilbert_java_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/DistilBert_java_Tokenizer/"

Mounted at /content/drive


# **Load and Test Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_java/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_java_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_java/best_model_distilbert_java.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_java_Tokenizer/best_model_distilbert_java_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/DistilBert_java/best_model_distilbert_java.zip
replace ./best_model_distilbert_java/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
best_model_distilbert_java = 'best_model_distilbert_java'
best_model_distilbert_java_tokenizer = 'best_model_distilbert_java_tokenizer'

In [ ]:
# Load the model and tokenizer from the checkpoint
model = DistilBertForSequenceClassification.from_pretrained(best_model_distilbert_java)
tokenizer = DistilBertTokenizer.from_pretrained(best_model_distilbert_java_tokenizer)

text = "Logits from the model output"
encoded_input = tokenizer(text, return_tensors='pt')
start_time = time.time()
output = model(**encoded_input)
end_time = time.time()
prediction_time = end_time - start_time
output

print("=============Prediction Time==============")
print(prediction_time)
print("==========================================")

In [ ]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")